In [1]:
import tensorflow as tf
import numpy as np

### 7.1load and save NDarray

我们可以直接使用save函数和load函数分别存储和读取。下面的例子创建了tensor x，并将其存在文件名同为x的文件里。

In [2]:
x = tf.ones(3)

In [3]:
x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>

将变量存储进文件

In [4]:
np.save('x.npy', x)

从文件中读取变量到内存

In [5]:
x2 = np.load('x.npy')
x2

array([1., 1., 1.], dtype=float32)

可以存储一列tensor并读回内存。

In [6]:
y = tf.zeros(4)
np.save('xy.npy',[x,y])
x2, y2 = np.load('xy.npy', allow_pickle=True)
(x2, y2)

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

存储并读取一个从字符串映射到tensor的字典。

In [7]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

In [9]:
mydict['x']

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>

### 7.2 load and save model parameters

我们还可以读写模型的参数。 为了演示方便，我们先创建一个多层感知机，并将其初始化。。

In [11]:
X = tf.random.normal((2,20))
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = self.dense1(x)    
        output = self.dense2(x)     
        return output

net = MLP()
Y = net(X)
Y

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.13624582, -0.2099525 , -0.4450876 , -0.23988141, -0.36348754,
        -0.4742679 , -0.1697901 , -0.07490838, -0.05205764, -0.19876322],
       [-0.46377593, -0.17872074, -0.5306553 ,  0.0645162 , -0.15479918,
         0.50395954, -0.18191639,  0.6380503 ,  0.45555204, -0.6638664 ]],
      dtype=float32)>

下面把该模型的参数存成文件，文件名为7.2saved_model.h5。

In [12]:
net.save_weights("7.2saved_model.h5")

接下来，我们再实例化一次定义好的多层感知机。与随机初始化模型参数不同，我们在这里直接读取保存在文件里的参数。

因为这两个实例都有同样的模型参数，那么对同一个输入X的计算结果将会是一样的。我们来验证一下。


In [13]:
net2 = MLP()
net2(X)
net2.load_weights("7.2saved_model.h5")
Y2 = net2(X)
Y2 == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

### GPU计算
- 对复杂的神经网络和大规模的数据来说，使用CPU来计算可能不够高效。我们将介绍如何使用单块NVIDIA GPU来计算。首先，需要确保已经安装好了至少一块NVIDIA GPU。然后，下载CUDA并按照提示设置好相应的路径

#### 注意：需要tensorflow-gpu

In [14]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print("可用的GPU：",gpus,"\n可用的CPU：", cpus)

2.3.1
可用的GPU： [] 
可用的CPU： [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


#### check available device

In [15]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13879886508633086992
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1300074299033839052
physical_device_desc: "device: XLA_CPU device"
]


#### specify device
- 使用tf.device()来指定特定设备(GPU/CPU)

In [ ]:
with tf.device('GPU:0'):
    a = tf.constant([1,2,3],dtype=tf.float32)
    b = tf.random.uniform((3,))
    print(tf.exp(a + b) * 2)